In [1]:
# group all 311 data by buildings, 
# get buildings' geoid using api and shapefiles,
# build a dictionary with key = (long, lat), and val= geoid
# process all 311 qualifying calls using the dictionary and retreive geoid

from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd
shapefile_directory = r'D:\dataGasLeaks\NYC_Census_Tracts_for_2010_US_Census-shp\NYC_Census_Tracts_for_2010_US_Census.shp'
shapefile = gpd.read_file(shapefile_directory)



In [ ]:
dff = pd.read_csv('D://dataGasLeaks//311_Service_Requests_updated.csv')
dff.shape
dff['count'] = 1
grouped_df = dff.groupby(['Created Date','Latitude', 'Longitude']).agg({'Borough': 'first', 'count': 'sum'})
grouped2_df = grouped_df.groupby(['Latitude', 'Longitude']).agg({'Borough': 'first', 'count': 'sum'})

## Dataframe with data from API: shorted block Fips to geoid, 
## Dataframe with data from shapefile: leftpad with zeros (6digits), find counties, create geoid

In [37]:
df1 = pd.read_csv('D://dataGasLeaks//311_Service_Requests_part13.csv')
df1 = df1[df1['ct'].notna()]
df1['ct'] = df1['ct'].astype(str).str[:11]

In [82]:
df2 = pd.read_csv('D://dataGasLeaks//311_Service_Requests_part9.csv')
df2 = df2[df2['ct'].notna()]

C:\Users\mskac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from shapely.geometry import Point, Polygon, MultiPolygon
import geopandas as gpd
import pandas as pd

shapefile_directory = r'D:\dataGasLeaks\table_2010_census_tracts_with_geoid\table_2010_census_tracts_with_geoid.shp'
shapefile = gpd.read_file(shapefile_directory)
shapefile

,cartodb_id,ntacode,shape_area,ntaname,shape_leng,boro_name,puma,ct2010,boro_code,cdeligibil,boro_ct201,ctlabel,block_fips,geometry
0,47,BK28,1216850.356,Bensonhurst West,4679.72972204,Brooklyn,4017,018600,3,I,3018600,186,360470186002001,"POLYGON ((-74.00203 40.61515, -74.00261 40.614..."
1,485,QN20,1221365.56287,Ridgewood,4479.23325647,Queens,4110,054500,4,E,4054500,545,360810545003000,"POLYGON ((-73.90973 40.70547, -73.91066 40.704..."
2,799,BX10,5709578.31711,Pelham Bay-Country Club-City Island,27385.1404928,Bronx,3703,027402,2,I,2027402,274.02,360050274021000,"POLYGON ((-73.81606 40.83603, -73.81641 40.835..."
3,892,QN45,14742052.7704,Douglas Manor-Douglaston-Little Neck,17888.3334867,Queens,4104,148300,4,I,4148300,1483,360811483004001,"POLYGON ((-73.73938 40.77601, -73.73991 40.775..."
4,1290,BK99,43216571.4155,park-cemetery-etc-Brooklyn,146017.76109,Brooklyn,4009,070203,3,I,3070203,702.03,360470702031009,"MULTIPOLYGON (((-73.86706 40.58209, -73.86769 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,2161,BK31,2167285.24055,Bay Ridge,7827.08344987,Brooklyn,4013,014200,3,E,3014200,142,360470142001009,"POLYGON ((-74.01886 40.62427, -74.01862 40.624..."
2161,2162,QN02,11459950.8969,Springfield Gardens North,15170.1260015,Queens,4112,029400,4,I,4029400,294,360810294005003,"POLYGON ((-73.77908 40.67787, -73.77862 40.677..."
2162,2163,SI05,21980474.2562,New Springville-Bloomfield-Travis,25335.4030572,Staten Island,3902,029103,5,I,5029103,291.03,360850291032005,"POLYGON ((-74.16242 40.60719, -74.16240 40.607..."
2163,2164,QN10,4259746.83217,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,9431.75241279,Queens,4114,093402,4,I,4093402,934.02,360810934021003,"POLYGON ((-73.83542 40.57696, -73.83576 40.576..."


In [3]:
shapefile.columns

Index(['cartodb_id', 'ntacode', 'shape_area', 'ntaname', 'shape_leng',
       'boro_name', 'puma', 'ct2010', 'boro_code', 'cdeligibil', 'boro_ct201',
       'ctlabel', 'block_fips', 'geometry'],
      dtype='object')

In [4]:
shapefile.ntacode

0       BK28
1       QN20
2       BX10
3       QN45
4       BK99
        ... 
2160    BK31
2161    QN02
2162    SI05
2163    QN10
2164    QN10
Name: ntacode, Length: 2165, dtype: object

In [126]:
# df2 - dataset where each row contains lat and long of a building
for ind, row in df2.iterrows():
    # for each building create a point (imported from shapely.geometry)
    p = Point(row['Longitude'], row['Latitude'])
    
    for ind_shape, shape in shapefile.iterrows():
        # check if the point belongs to a geometry of a current row in a shapefile
        if p.within(shape['geometry']):
            df2.at[ind, 'ct'] = str(shape['block_fips'])
            break

In [129]:
df = pd.concat([df1, df2[:200000], df2[500000:]])

In [130]:
df.to_csv(r'D:\dataGasLeaks\part1_ready.csv')

# Include 311 complaints into data

In [202]:
df = pd.read_csv(r'D:\dataGasLeaks\final_311_data.csv')
df_original = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data.csv')

In [203]:
df = df.rename(columns={'year': 'incident_year', 'count': '311complaints'})
df = df.groupby(['geoid', 'incident_year']).agg({'311complaints': 'sum'})

df_new = pd.merge(df_original, df, on=['geoid','incident_year'], how='left')
df_new['population'] = df_new['gas_leaks']/df_new['gas_leaks_per_person']

In [215]:
df_new['311compl_per_person'] = df_new['311complaints']/df_new['population']
df_new.drop(['Unnamed: 0', '311complaints','population'], axis = 1, inplace = True)

In [217]:
df_new.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data1.csv')

## All years data

In [231]:
df = pd.read_csv(r'D:\dataGasLeaks\final_311_data.csv')
df_original = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data.csv')
df_goal = pd.read_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data_all_years.csv')

In [233]:
df_original['population'] = df_original['gas_leaks']/df_original['gas_leaks_per_person']
df_original = df_original.groupby('geoid').agg({'population': 'sum'})

In [234]:
df = df.groupby('geoid').agg({'count': 'sum'})

df_new = pd.merge(df_original, df, on=['geoid'], how='left')
df_new['311compl_per_person'] = df_new['count']/df_new['population']

In [235]:
df_new = df_new.reset_index()
df_new.columns

Index(['geoid', 'population', 'count', '311compl_per_person'], dtype='object')

In [236]:
df_new.drop(['count','population'], axis = 1, inplace = True)
df_new.head()

,geoid,311compl_per_person
0,36005000200,0.261595
1,36005000400,0.191689
2,36005001600,0.145979
3,36005001900,0.644376
4,36005002000,0.141996


In [237]:
df_goal = pd.merge(df_goal, df_new, on=['geoid'], how='left')
df_goal

,Unnamed: 0,%housh_grandp_resp_for_grandch,avg_houshold_size,avg_bldg_age,gas_leaks,gas_leaks_per_person,geoid,lonely_housholder%,lonely_housholder_over65%,mean_houshold_income,median_age,nonfamily_housholds%,not_us_citizen%,total_housing_units,unemployed%,vacant_housing_units%,311compl_per_person
0,0,5.553494,3.730000,63.858957,38,0.001258,36005000200,17.439703,8.188002,75457.16667,38.300000,20.519481,13.337968,8880,4.217565,8.952703,0.261595
1,1,1.970487,3.031667,52.749939,49,0.001425,36005000400,16.311743,5.876116,85459.33333,38.016667,21.525139,7.444241,12399,3.986740,8.726510,0.191689
2,2,0.826587,2.815000,56.219298,99,0.002862,36005001600,27.158074,13.583298,47136.66667,37.466667,27.942054,16.517315,12290,3.173961,4.515867,0.145979
3,3,2.451161,2.425000,83.272222,38,0.002377,36005001900,37.670475,1.787689,47365.33333,31.333333,47.806856,15.832604,5763,8.438634,5.847649,0.644376
4,4,4.998347,2.981667,75.921555,314,0.005806,36005002000,29.499614,13.126860,34932.00000,33.416667,30.342775,9.354833,19106,5.571479,5.024600,0.141996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,2046,0.898835,2.738333,46.795396,46,0.001538,36085030301,28.313308,9.933046,77313.50000,35.233333,32.844171,8.059123,11350,3.501204,3.938326,0.158608
2047,2047,1.448905,3.231667,34.557914,38,0.001002,36085030302,13.219126,4.321145,85669.83333,35.483333,16.372624,13.835732,12509,2.778217,6.203533,0.178475
2048,2048,2.962029,2.600000,43.612317,97,0.005873,36085031901,24.452497,15.976052,35690.00000,30.050000,24.452497,9.676052,7213,3.645171,12.006100,0.189949
2049,2049,2.276439,2.886667,56.334072,22,0.000774,36085031902,21.355655,6.900776,54850.66667,28.733333,24.938751,9.278024,11354,2.469918,13.722036,0.280100


In [239]:
df_goal.to_csv(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\important_(used_in_app)\Merged_asc_fdny_data_all_years1.csv')

type                                        FeatureCollection
features    {'type': 'Feature', 'properties': {'ntacode': ...
Name: 0, dtype: object

In [ ]:
geojson = pd.read_json(r'C:\Users\mskac\machineLearning\GasLeakConEd\data\processed\2010 Census Tracts with geoid.geojson')
geojson.iloc[0]

In [248]:
geojson.iloc[0]['features']

{'type': 'Feature',
 'properties': {'ntacode': 'SI22',
  'shape_area': '2497009.71359',
  'ntaname': 'West New Brighton-New Brighton-St. George',
  'shape_leng': '7729.01679383',
  'boro_name': 'Staten Island',
  'puma': '3903',
  'ct2010': '000900',
  'boro_code': '5',
  'cdeligibil': 'E',
  'boro_ct2010': '5000900',
  'ctlabel': '9',
  'block_fips': '360850009001003'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-74.07920577013245, 40.64343078374567],
     [-74.07913504647637, 40.64333789954809],
     [-74.07819773831595, 40.64211481672276],
     [-74.07884632409665, 40.641995568528166],
     [-74.0791223401088, 40.64194431928626],
     [-74.07965266950632, 40.641855351552806],
     [-74.08055489346589, 40.64169150377349],
     [-74.08148001617707, 40.64155046162583],
     [-74.08130714145031, 40.64074191709323],
     [-74.08110049694088, 40.639922825339106],
     [-74.08191207797016, 40.6398059117602],
     [-74.08194326513642, 40.63993327788856],
     [-74.0820464807

In [251]:
import codecs
with codecs.open("C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/2010 Census Tracts with geoid.geojson", 'r', 'utf8') as data_file:    
    data = json.load(data_file)
for element in data['features']:
    element['properties']['geoid'] = element['properties']['block_fips'][:11]  

with open('C:/Users/mskac/machineLearning/GasLeakConEd/data/processed/CT_shapes_with_geoids.geojson', 'w') as fp:
    json.dump(data, fp)

In [2]:
import pandas as pd
import json
import requests
from urllib.request import urlopen
import csv

key = '05fcc51d368d71bce9d7627d3097d978'
projectId = '82d5e815'

In [3]:
nodata = pd.read_csv(r'D:\dataGasLeaks\not_finished.csv')
len(nodata)

C:\Users\mskac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


798452

In [36]:
nodata = nodata[nodata['geoid'].notna()]

In [4]:
for ind in range (400000,450000):
    long = nodata.iloc[ind]['Longitude']
    lat = nodata.iloc[ind]['Latitude']
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={0}&longitude={1}&showall=true&format=json'.format(lat, long)
    response = requests.get(url)
    dataJSON = response.json()
    data = dataJSON["Block"]['FIPS']
    nodata.at[ind, 'geoid'] = data
print('done')

done


In [7]:
for ind in range (452682,500000):
    long = nodata.iloc[ind]['Longitude']
    lat = nodata.iloc[ind]['Latitude']
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={0}&longitude={1}&showall=true&format=json'.format(lat, long)
    response = requests.get(url)
    dataJSON = response.json()
    data = dataJSON["Block"]['FIPS']
    nodata.at[ind, 'geoid'] = data
print('done')

done


In [8]:
nodata.to_csv(r'D:\dataGasLeaks\not_finished2.csv')

In [9]:
for ind in range (500000,550000):
    long = nodata.iloc[ind]['Longitude']
    lat = nodata.iloc[ind]['Latitude']
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={0}&longitude={1}&showall=true&format=json'.format(lat, long)
    response = requests.get(url)
    dataJSON = response.json()
    data = dataJSON["Block"]['FIPS']
    nodata.at[ind, 'geoid'] = data
print('done')

done


In [10]:
nodata.to_csv(r'D:\dataGasLeaks\not_finished2.csv')

In [11]:
for ind in range (550000,600000):
    long = nodata.iloc[ind]['Longitude']
    lat = nodata.iloc[ind]['Latitude']
    url = 'https://geo.fcc.gov/api/census/block/find?latitude={0}&longitude={1}&showall=true&format=json'.format(lat, long)
    response = requests.get(url)
    dataJSON = response.json()
    data = dataJSON["Block"]['FIPS']
    nodata.at[ind, 'geoid'] = data
print('done')

done


In [37]:
nodata.to_csv(r'D:\dataGasLeaks\not_finished2.csv')

In [ ]:
nodata = nodata[]

In [38]:
data1 = pd.read_csv(r'D:\dataGasLeaks\not_finished1.csv')
data2 = pd.read_csv(r'D:\dataGasLeaks\not_finished2.csv')

C:\Users\mskac\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
len(data1[data1['geoid'].notna()])

400000

In [40]:
len(data2[data2['geoid'].notna()])

242707

In [41]:
data_missing = pd.concat([data1, data2])

In [42]:
data_missing.to_csv(r'D:\dataGasLeaks\data_missing.csv')